### Define a Base class for Working with Automation Rules 

In [1]:
import sys

In [2]:
class Rule:
    """A Simple IoT Automation Rule Class"""
    
    def __init__(self, sensorID):
        self.sensorID = sensorID
    
    def predicate(self, sensorValue):
        return True
    
    def do(self, sensorValue):
        try:
            if self.predicate(sensorValue):
                self.action(sensorValue)
        except:
            print "Unexpected error:", sys.exc_info()[0]
                    
    def action(self, sensorValue):
        print("Generic Rule activiate on " + 
              self.sensorID + " with senor value " + str(sensorValue))    

### Define a Subclass Rule Class that can be Instantiated

In [3]:
class TemperatureOver20Rule(Rule):
    def predicate(self, sensorValue):
        return sensorValue > 20
    
    def action(self, sensorValue):
        print("Temperature Over 20 Rule activated on " + self.sensorID + " with senor value " + str(sensorValue))

In [4]:
class TemperatureUnder15Rule(Rule):
    def predicate(self, sensorValue):
        return sensorValue < 15
    
    def action(self, sensorValue):
        print("Temperature Under 15 Rule activated on " + self.sensorID + " with senor value " + str(sensorValue))

### Instantiate the Rules

In [5]:
r1 = Rule("temperature1")
r2 = TemperatureUnder15Rule("temperature")
r3 = TemperatureOver20Rule("temperature")
r4 = Rule("temperature4")

In [6]:
rules = r1, r2, r3, r4

Return an array of results from each rules predicate function

### Define Helper Functions

The first helper function that we will define will take the entire list of automation rules and filter it based on sensorID. Becasue we are listening to JSON object that contain a **sensorID**, a **value** and a **timestamp**, this enables us to filter the automation rules to only include the rules that apply to the sensorID of the incoming data.

In [7]:
def filterBySensorId(sensorID, rules):
    "Filter a list of rules by sensorID"
    return [rule for rule in rules if rule.sensorID == sensorID]

The second filter that we will define will take a list of automation rules and execute each of their **predicate** functions and return a list of rules that should be executed on the incoming data.

In [8]:
def filterByPredicate(sensorData, rules):
    "Filter a list of rules by its predicate"
    [rule for rule in rules if rule.predicate(sensorData) == True]

In [9]:
print filterByPredicate(20, rules)

None


Let's test this helper to verify that it works as expected.

In [10]:
filterBySensorId("temperature1", rules)

[<__main__.Rule instance at 0x7fb4ec540248>]

In [11]:
filterBySensorId("temperature2", rules)

[]

In [12]:
filterBySensorId("temperature3", rules)

[]

In [13]:
filterBySensorId("temperature4", rules)

[<__main__.Rule instance at 0x7fb4ec540518>]

In [14]:
[r.predicate(20) for r in rules]

[True, False, False, True]

In [15]:
[r.do(21) for r in rules]

Generic Rule activiate on temperature1 with senor value 21

Temperature Over 20 Rule activated on temperature with senor value 21

Generic Rule activiate on temperature4 with senor value 21


[None, None, None, None]

## Helper Functions to Encode/Decode JSON

In [16]:
import json

In [17]:
sampleData = '{"sensor_id":"temperature3","value":17,"timestamp":1513807710949}'

In [18]:
try:
    parsed_json = json.loads(sampleData)
except:
     print("Unexpected error:", sys.exc_info()[0])

In [19]:
parsed_json['value']

17

In [20]:
[r.do(parsed_json['value']) 
  for r in filterBySensorId(
                  parsed_json['sensor_id'], rules
  )]

[]

## Setup the MQTT subscription of the Automation Service

In [21]:
import paho.mqtt.client as mqtt

In [22]:
def on_connect(mqttc, obj, flags, rc):
    print("rc: " + str(rc))

In [23]:
def on_message(mqttc, obj, msg):
    print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    try:
        parsed_json = json.loads(msg.payload)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        
    [r.do(parsed_json['value']) for r in filterBySensorId(parsed_json['sensor_id'], rules)]

In [24]:
def on_subscribe(mqttc, obj, mid, granted_qos):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

In [25]:
def on_log(mqttc, obj, level, string):
    print(string)

In [26]:
mqttc = mqtt.Client()
mqttc.on_message = on_message
mqttc.on_connect = on_connect
mqttc.on_subscribe = on_subscribe

# Uncomment to enable debug messages
# mqttc.on_log = on_log

In [27]:
mqttc.connect("localhost", 1883, 60)

0

In [ ]:
mqttc.subscribe("sensors/temperature/data")

(0, 1)

In [ ]:
mqttc.loop_forever()

rc: 0
Subscribed: 1 (0,)
sensors/temperature/data 0 {"sensor_id":"temperature","value":24,"timestamp":1513811032566}

Temperature Over 20 Rule activated on temperature with senor value 24

sensors/temperature/data 0 {"sensor_id":"temperature","value":21,"timestamp":1513811033570}

Temperature Over 20 Rule activated on temperature with senor value 21

sensors/temperature/data 0 {"sensor_id":"temperature","value":19,"timestamp":1513811034572}
sensors/temperature/data 0 {"sensor_id":"temperature","value":21,"timestamp":1513811035573}

Temperature Over 20 Rule activated on temperature with senor value 21

sensors/temperature/data 0 {"sensor_id":"temperature","value":24,"timestamp":1513811036574}

Temperature Over 20 Rule activated on temperature with senor value 24

sensors/temperature/data 0 {"sensor_id":"temperature","value":15,"timestamp":1513811037576}
sensors/temperature/data 0 {"sensor_id":"temperature","value":17,"timestamp":1513811038578}
sensors/temperature/data 0 {"sensor_id":"te